<a href="https://colab.research.google.com/github/julianorinaldi/CAIA003-DAT003-DataMining/blob/main/Atividade1/Exercicio1-2/Exerc%C3%ADcio_1%5BTwitter%5D%5BJuliano_Rinaldi_CAIA003_DAT003%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   i- Instale todas as bibliotecas mencionadas na apresentação teórica sobre coleta de dados.
*   ii- Crie uma aplicação no Twitter. Em seguida, gere os credenciais necessárias para acessar dados da
API do Twitter.

1.   Faça uma adaptação no coletor de tweets para coletar qualquer tipo de tweet compartilhado no Brasil. 
Dica: veja a documentação da biblioteca TwitterApi. Faça uma coleta por alguns minutos, salvando o resultado em um arquivo ou em um banco de dados.

2.   Com base no dataset gerado na questão 1, faça:
*   a) Separe todas as hashtags usadas nos Tweets.
*   b) Descubra a porcentagem de tweets com hashtags.
*   c) Identifique a hashtag mais popular.


# Instalação dos Pacotes Twitter

In [1]:
!pip install TwitterAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Importando Bibliotecas

In [2]:
from TwitterAPI import TwitterAPI

# Desenvolvimento

In [3]:
file_name="./credentials.txt"
f= open(file_name,"w")
f.write("consumer_key=dnCKtBcAezQStwPe3Mwg6QKRe")
f.write("\n")
f.write("consumer_secret=DGKnckbRUu0u2x0FtvA5BM57LagL9sMm8eHYPmWRNgfCKggSOh")
f.write("\n")
f.write("access_token_key=233363596-xagz6EQq1BJsWIQatXbPkQA7VB50MYpBZm7KdcEM")
f.write("\n")
f.write("access_token_secret=W8dTMjdeAiqbybIZsqI0eWaC0sUQSJhGFAwcHtxMeyInK")
f.close()

!cat credentials.txt

consumer_key=dnCKtBcAezQStwPe3Mwg6QKRe
consumer_secret=DGKnckbRUu0u2x0FtvA5BM57LagL9sMm8eHYPmWRNgfCKggSOh
access_token_key=233363596-xagz6EQq1BJsWIQatXbPkQA7VB50MYpBZm7KdcEM
access_token_secret=W8dTMjdeAiqbybIZsqI0eWaC0sUQSJhGFAwcHtxMeyInK

In [4]:
# from TwitterAPI import TwitterAPI, TwitterOAuth, TwitterRequestError, TwitterConnectionError

# QUERY = 'UTFPR'
# EXPANSIONS = 'author_id,referenced_tweets.id,referenced_tweets.id.author_id,in_reply_to_user_id,attachments.media_keys,attachments.poll_ids,geo.place_id,entities.mentions.username'
# TWEET_FIELDS='author_id,conversation_id,created_at,entities,geo,id,lang,public_metrics,source,text'
# USER_FIELDS='created_at,description,entities,location,name,profile_image_url,public_metrics,url,username'
# AMOUNT_TWITTERS = 1

# try:
#   o = TwitterOAuth.read_file(file_name=file_name)
#   api = TwitterAPI(o.consumer_key, o.consumer_secret, o.access_token_key, o.access_token_secret, api_version='2')
#   r = api.request('tweets/search/recent', {
#     'query':QUERY, 
#     'tweet.fields':TWEET_FIELDS,
#     'expansions':EXPANSIONS,
#     'user.fields': USER_FIELDS})
  
#   count = 0
#   for item in r:
#     print(json.dumps(item, indent=2))
#     count += 1
#     print(f'################# Coletado  [{count}] Twitters #########################')
#     if count >= AMOUNT_TWITTERS:
#       break

#   print('\nINCLUDES')
#   #print(json.dumps(r.json()['includes'], indent=2))
#   print(r.json()['includes'])

#   print('\nQUOTA')
#   print(json.dumps(r.get_quota(), indent=2))
#   #print(r.get_quota())

# except TwitterRequestError as e:
# 	print(e.status_code)
# 	for msg in iter(e):
# 		print(msg)

# except TwitterConnectionError as e:
# 	print(e)

# except Exception as e:
# 	print(e)

In [5]:
from TwitterAPI import TwitterAPI, TwitterOAuth, TwitterRequestError, TwitterConnectionError, HydrateType, OAuthType
import json

# Filtros
QUERY = 'UTFPR'
EXPANSIONS = 'author_id'
TWEET_FIELDS='author_id,text,source'
USER_FIELDS='created_at,name,username,entities'

#Quantidade de Twitters deseja coletar
AMOUNT_TWITTERS = 2

try:
	o = TwitterOAuth.read_file(file_name=file_name)
	api = TwitterAPI(o.consumer_key, o.consumer_secret, auth_type=OAuthType.OAUTH2, api_version='2')

	# Responsável pela limpeza de todas as regras de filtros
	r = api.request('tweets/search/stream/rules', method_override='GET')
	if r.status_code != 200:
		exit()
	
	rules=r.json()
	ids = list(map(lambda rule: rule["id"], rules["data"]))
	payload = {"delete": {"ids": ids}}
	r = api.request('tweets/search/stream/rules', payload)
	print(f'[{r.status_code}] REGRAS DELETADAS: {json.dumps(r.json(), indent=2)}\n')
	if r.status_code != 200:
		exit()
	
	# Adicionar regras ao Stream

	r = api.request('tweets/search/stream/rules', {'add': [{'value':QUERY}]})
	print(f'[{r.status_code}] REGRAS ADICIONADAS: {json.dumps(r.json(), indent=2)}\n')
	if r.status_code != 201:
		exit()

	# Lista regras do Stream

	r = api.request('tweets/search/stream/rules', method_override='GET')
	print(f'[{r.status_code}] REGRAS: {json.dumps(r.json(), indent=2)}\n')
	if r.status_code != 200:
		exit()

	# Inicia abertura do Stream para pegar Twitters

	r = api.request('tweets/search/stream', {
			'expansions': EXPANSIONS,
			'tweet.fields': TWEET_FIELDS,
			'user.fields': USER_FIELDS
		},
		hydrate_type=HydrateType.APPEND)

	print(f'[{r.status_code}] INICIANDO...')
	if r.status_code != 200: 
		exit()
 
	count = 0
	for item in r:
		print(json.dumps(item, indent=2))
		count += 1
		print(f'################# Coletado  [{count}] Twitters #########################')
		with open("./tweets_store.txt", 'w') as json_file:
			json.dump(item, json_file, 
													indent=4,  
													separators=(',',': '))
		if count >= AMOUNT_TWITTERS:
			break

except Exception as e:
	print(e)

!cat ./tweets_store.txt

[200] REGRAS DELETADAS: {
  "meta": {
    "sent": "2022-06-12T00:04:20.721Z",
    "summary": {
      "deleted": 1,
      "not_deleted": 0
    }
  }
}

[201] REGRAS ADICIONADAS: {
  "data": [
    {
      "value": "UTFPR",
      "id": "1535774970239782913"
    }
  ],
  "meta": {
    "sent": "2022-06-12T00:04:20.902Z",
    "summary": {
      "created": 1,
      "not_created": 0,
      "valid": 1,
      "invalid": 0
    }
  }
}

[200] REGRAS: {
  "data": [
    {
      "id": "1535774970239782913",
      "value": "UTFPR"
    }
  ],
  "meta": {
    "sent": "2022-06-12T00:04:21.023Z",
    "result_count": 1
  }
}

[200] INICIANDO...


KeyboardInterrupt: ignored